# Setup

Perform all necessary imports up front

In [ ]:
# Python standard imports
from pprint import pprint

# armory-library imports
import armory.engine
import armory.evaluation
import armory.utils

# armory-examples imports
import armory.examples.object_detection.license_plates_yolos as license_plates
from armory.examples.utils.display import display_object_detection_results

In [ ]:
batch_size = 2
num_batches = 2

We define some runtime parameters here. These can be overwritten externally
via papermill (e.g., `-p batch_size 4`). Modify these as necessary based on your
host resource constraints.

# Define the Evaluation

In [ ]:
evaluation = armory.evaluation.Evaluation(
    name="object-detection-license-plates",
    description="Object detection of license-plates",
    author="TwoSix",
)

## Model

From our YOLOS license plates example, we will load a model from HuggingFace
that has already been fine-tuned on the roboflow license plates dataset. We also
wrap this model in an Adversarial Robustness Toolbox (ART) estimator so that we
can use an ART attack against the model.

In [ ]:
with evaluation.autotrack():
    model, art_estimator = license_plates.load_model()
evaluation.use_model(model)

## Dataset

From our YOLOS license plates example, we will load the roboflow license plates
dataset from HuggingFace.

In [ ]:
with evaluation.autotrack():
    dataset = license_plates.load_dataset(batch_size=batch_size, shuffle=False)
evaluation.use_dataset(dataset)

## Attack

From our YOLOS license plates example, we create a robust DPatch attack using
the Adversarial Robustness Toolbox (ART).

In [ ]:
with evaluation.autotrack():
    attack = license_plates.create_attack(art_estimator)

## Metrics

From our YOLOS license plates example, we create the metrics to be collected
during the evaluation. These include an L-infinity norm distance between
unperturbed and perturbed input, and a mean average precision between the
natural objects and the predicted objects.

In [ ]:
evaluation.use_metrics(
    license_plates.create_metrics()
)

## Exporters

From our YOLOS license plates example, we create the exporters used to record
sample images during the evaluation.

In [ ]:
evaluation.use_exporters(
    license_plates.create_exporters(model, export_every_n_batches=1)
)

## Evaluation Chains

We will define two perturbation chains: `benign` and `attack`. The benign chain
does not apply any perturbations to the data, giving us the intrinsic
performance of the model. The attack chain will give us the performance of the
model under adversarial attack.

In [ ]:
with evaluation.add_chain("benign"):
    pass

with evaluation.add_chain("attack") as chain:
    chain.add_perturbation(attack)

# Execute the Evaluation

We create an evaluation engine which will handle the application of all
perturbations, obtaining predictions from the model, collecting metrics, and
exporting of samples.

In [ ]:
engine = armory.engine.EvaluationEngine(
    evaluation,
    limit_test_batches=num_batches,
)
results = engine.run()

In [ ]:
pprint(results)

In [ ]:
display_object_detection_results(
    chains={name: res["run_id"] for name, res in results.items()},
    batch_idx=0,
    batch_size=batch_size,
)